In [4]:
!pip -q install selenium

Python(50915) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [6]:
import time
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [19]:
def setup_filter_and_parse(start_url):
    driver = webdriver.Chrome()
#   driver.set_window_size(800, 800)
    driver.get(start_url)

    # filter_element = driver.find_element(By.ID, 'filter_id')
    # filter_element.send_keys(filter_criteria)

    # apply_button = driver.find_element(By.ID, 'apply_button_id')
    # apply_button.click()
    
    toggle_button_no = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'toggle-button-no'))
    )
    toggle_button_yes = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'toggle-button-yes'))
    )

    #if toggle_button_no.value_of_css_property('display') == 'block':
    toggle_button_no.click()
    
#     ilter_button = WebDriverWait(driver, 1).until(
#             EC.element_to_be_clickable((By.CLASS_NAME, 'btn-small-outline-v02'))
#         )
#     filter_button.click()
    
    genre_checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'genre_prose'))
        )
    if not genre_checkbox.is_selected():
        genre_checkbox.click()

    lang_checkbox = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'lang_he'))
    )
    if not lang_checkbox.is_selected():
        lang_checkbox.click()

    #WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CLASS_NAME, 'result_class_name')))
    #WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CLASS_NAME, 'result_class_name')))
    parse_all_pages(driver)

    driver.quit()

def parse_page(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', href=True)
    for link in links:
        if "/read/" in link['href']:
            href = link['href']
            text_url = requests.compat.urljoin(driver.current_url, href)
            get_text_from_link(text_url, driver)

def get_text_from_link(url, driver):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    title_div = soup.find('div', class_='headline-1-v02')
    if title_div:
        title = title_div.get_text().strip()
    else:
        title = "Untitled"

    text_div = soup.find('div', {'id': 'actualtext'})
    if text_div:
        text = text_div.get_text(separator='\n').strip()
        save_text_to_file(title, text)

def save_text_to_file(title, text):
    if not os.path.exists('pure'):
        os.makedirs('pure')

    safe_title = "".join(x for x in title if x.isalnum() or x in "._- ")
    base_file_path = os.path.join('pure', f"{safe_title}.txt")

    file_path = base_file_path
    count = 1
    while os.path.exists(file_path):
        file_path = os.path.join('pure', f"{safe_title} ({count}).txt")
        count += 1

    with open(file_path, 'w') as file:
        file.write(text)

def next_page(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'by-button-v02 browse_paging'))
        )
        next_button.click()
        return True
    except:
        return False

def parse_all_pages(driver):
    while True:
        parse_page(driver)
        if not next_page(driver):
            break

In [82]:
def apply_filters(driver):
    try:
        toggle_button_no = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'toggle-button-no'))
        )
        if toggle_button_no.value_of_css_property('display') == 'block':
            toggle_button_no.click()

        genre_checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'genre_prose'))
        )
        if not genre_checkbox.is_selected():
            genre_checkbox.click()

        lang_checkbox = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'lang_he'))
        )
        if not lang_checkbox.is_selected():
            lang_checkbox.click()

        time.sleep(5)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'result_class_name'))
        )

    except Exception as e:
        print(e)

def parse_page(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    links = soup.find_all('a', href=True)
    for link in links:
        if "/read/" in link['href']:
            href = link['href']
            text_url = driver.current_url.rsplit('/', 1)[0] + href
            get_text_from_link(text_url, driver)

def get_text_from_link(url, driver):
    current_window = driver.current_window_handle

    driver.execute_script("window.open(arguments[0]);", url)
    driver.switch_to.window(driver.window_handles[-1])

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        title_div = soup.find('div', class_='headline-1-v02')
        title = title_div.get_text().strip() if title_div else "Untitled"

        text_div = soup.find('div', {'id': 'actualtext'})
        if text_div:
            text = text_div.get_text(separator='\n').strip()
            save_text_to_file(title, text)

    finally:
        driver.close()
        driver.switch_to.window(current_window)

def save_text_to_file(title, text):
    if not os.path.exists('pure'):
        os.makedirs('pure')

    safe_title = "".join(x for x in title if x.isalnum() or x in "._- ")
    base_file_path = os.path.join('pure', f"{safe_title}.txt")

    file_path = base_file_path
    count = 1
    while os.path.exists(file_path):
        file_path = os.path.join('pure', f"{safe_title} ({count}).txt")
        count += 1

    with open(file_path, 'w') as file:
        file.write(text)

def switch_to_next_page(driver):
    try:
        time.sleep(5)

        next_buttons = driver.find_elements(By.XPATH, "//*[contains(text(), 'לדף הבא')]")

        if not next_buttons:
            return False

        for button in next_buttons:
            try:
                button.click()

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'result_class_name'))
                )
                return True
            except Exception as e:
                print(e)
                continue

    except Exception as e:
        print(e)
    return True

In [18]:
start_url = 'https://benyehuda.org/works'
setup_filter_and_parse(start_url)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001008bfac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001008b8314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001003204b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001002fb994 core::str::slice_error_fail::ha0e52dbcb60e6bae + 3780
4   chromedriver                        0x000000010038a890 cxxbridge1$string$len + 524388
5   chromedriver                        0x000000010039d76c cxxbridge1$string$len + 601920
6   chromedriver                        0x00000001003590b0 cxxbridge1$string$len + 321668
7   chromedriver                        0x0000000100359d00 cxxbridge1$string$len + 324820
8   chromedriver                        0x000000010088ae08 cxxbridge1$str$ptr + 3435328
9   chr

In [44]:
start_url = 'https://benyehuda.org/works'
apply_filters_and_switch_page(start_url)

Toggle button is visible, clicking it
Clicking genre checkbox
Clicking language checkbox
Waiting for 5 seconds
Found 2 buttons with text 'לדף הבא'
Trying to click the next page button
Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="9418" data-search-after-page="2" data-search-after-reverse="false" data-search-after-value="אגדת שלושה וארבעה נוסח אחד">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x0000000104dcbac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000104dc4314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x000000010482c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x000000010487612c cxxbridge1$string$len + 391424
4   chromedriver         

In [83]:
start_url = 'https://benyehuda.org/works'

driver = webdriver.Chrome()
driver.set_window_size(1200, 800)
driver.get(start_url)

apply_filters(driver)
parse_page(driver)

while True:
    success = switch_to_next_page(driver)
    if not success:
        break
    parse_page(driver)

print("Finished parsing all pages.")

Toggle button is visible, clicking it
Clicking genre checkbox
Clicking language checkbox
Waiting for 5 seconds
An error occurred while applying filters: Message: 
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100d9c898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100dd60fc cxxbridge1$string$len + 604368
5   chromedriver                        0x0000000100d910b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000100d91d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x00000001012c2e08 cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x00000001012c6120 cxxbridge1$str$ptr + 3448408
9   chromedriver        

Could not click the button: Message: 
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100d9c898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100dd60fc cxxbridge1$string$len + 604368
5   chromedriver                        0x0000000100d910b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000100d91d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x00000001012c2e08 cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x00000001012c6120 cxxbridge1$str$ptr + 3448408
9   chromedriver                        0x00000001012aa17c cxxbridge1$str$ptr + 3333812
10  chromedriver                        0x00000001012c69e0 cx

Waiting for 5 seconds before attempting to switch page
Found 2 buttons with text 'לדף הבא'
Trying to click the next page button
Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="52031" data-search-after-page="6" data-search-after-reverse="false" data-search-after-value="אנו נוסעים למירון">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848

Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="12034" data-search-after-page="8" data-search-after-reverse="false" data-search-after-value="בבית הרידקציה">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848
5   chromedriver                        0x0000000100d9e6c0 cxxbridge1$string$len + 376468
6   chromedriver                        0

Could not click the button: Message: 
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100d9c898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100dd60fc cxxbridge1$string$len + 604368
5   chromedriver                        0x0000000100d910b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000100d91d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x00000001012c2e08 cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x00000001012c6120 cxxbridge1$str$ptr + 3448408
9   chromedriver                        0x00000001012aa17c cxxbridge1$str$ptr + 3333812
10  chromedriver                        0x00000001012c69e0 cx

...

Waiting for 5 seconds before attempting to switch page
Found 2 buttons with text 'לדף הבא'
Trying to click the next page button
Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="5162" data-search-after-page="55" data-search-after-reverse="false" data-search-after-value="פחד בלילה">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848
5   chr

Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="23126" data-search-after-page="57" data-search-after-reverse="false" data-search-after-value="צילי צל צל צל – הגמד העצל מקור">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848
5   chromedriver                        0x0000000100d9e6c0 cxxbridge1$string$len + 376468
6   chromedriver       

Could not click the button: Message: 
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100d9c898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100dd60fc cxxbridge1$string$len + 604368
5   chromedriver                        0x0000000100d910b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000100d91d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x00000001012c2e08 cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x00000001012c6120 cxxbridge1$str$ptr + 3448408
9   chromedriver                        0x00000001012aa17c cxxbridge1$str$ptr + 3333812
10  chromedriver                        0x00000001012c69e0 cx

Waiting for 5 seconds before attempting to switch page
Found 2 buttons with text 'לדף הבא'
Trying to click the next page button
Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="46041" data-search-after-page="62" data-search-after-reverse="false" data-search-after-value="שכן טוב">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848
5   chro

Could not click the button: Message: element click intercepted: Element <button class="by-button-v02 browse_paging" data-search-after-id="21118" data-search-after-page="64" data-search-after-reverse="false" data-search-after-value="שעה אחת ברחוב">...</button> is not clickable at point (725, 16). Other element would receive the click: <div class="col-6 col-lg-4">...</div>
  (Session info: chrome=131.0.6778.85)
Stacktrace:
0   chromedriver                        0x00000001012f7ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012f0314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d584b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da212c cxxbridge1$string$len + 391424
4   chromedriver                        0x0000000100da077c cxxbridge1$string$len + 384848
5   chromedriver                        0x0000000100d9e6c0 cxxbridge1$string$len + 376468
6   chromedriver                        